<a href="https://colab.research.google.com/github/JanMeow/productDataOptimization/blob/main/Chapter2_data_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GnM Dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!git clone https://github.com/JanMeow/productDataOptimization.git

Cloning into 'productDataOptimization'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 719.42 KiB | 12.85 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import time
from functools import wraps
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"⏱️ Function `{func.__name__}` took {end - start:.4f} seconds")
        return result
    return wrapper

In [ ]:
import pandas as pd
import numpy as np

# Load Excel and sheet names
def load_excel_data(excel_file):
  sheet_data = {}
  xls = pd.ExcelFile(excel_file)
  sheet_keys = xls.sheet_names
  for sheet_name in sheet_keys:
      df_raw = pd.read_excel(excel_file, sheet_name=sheet_name, header=None)

      # Extract material names from row 3
      raw_material_names = df_raw.iloc[3, 1:].dropna().tolist()
      raw_material_names = [str(name).strip() for name in raw_material_names]
      start_col = 1

      # Track seen keys and build unique names
      name_counts = {}
      final_material_names = []

      for name in raw_material_names:
          if name not in name_counts:
              name_counts[name] = 1
              final_material_names.append(name)
          else:
              name_counts[name] += 1
              final_material_names.append(f"{name}{name_counts[name]}")

      # --- Extract df_0 (properties like Lambda-Wert) ---
      properties_rows = [4, 5]
      properties = []

      for row in properties_rows:
          row_label = df_raw.iloc[row, 0]
          values = df_raw.iloc[row, start_col::2].values[:len(final_material_names)]
          properties.append([row_label] + list(values))

      df_0 = pd.DataFrame(properties, columns=["Property"] + final_material_names)

      # Convert each column (excluding 'Property') into a numpy array
      data_0 = {}
      for material in df_0.columns[1:]:
        values = df_0[material].to_numpy()
        data_0[material] = values

      # --- Extract df_1 as dict of np arrays ---
      data_1 = {}
      name_counts = {}  # Reset to reassign same renaming logic for df_1

      for i, base_name in enumerate(raw_material_names):
          col_thickness = start_col + i * 2
          col_price = col_thickness + 1

          # Get group of entries
          group = []
          for row in range(8, df_raw.shape[0]):
              dicke = df_raw.iloc[row, col_thickness]
              preis = df_raw.iloc[row, col_price]
              if isinstance(dicke, str) or isinstance(preis, str):
                  continue
              if pd.notna(dicke) and pd.notna(preis):
                  group.append([dicke, preis])

          # Rename key consistently
          if base_name not in name_counts:
              name_counts[base_name] = 1
              final_key = base_name
          else:
              name_counts[base_name] += 1
              final_key = f"{base_name}{name_counts[base_name]}"

          if group:
              data_1[final_key] = np.array(group)

      # Store result
      sheet_data[sheet_name] = {
          "properties": data_0,
          "variants": data_1
      }
  return sheet_data


In [ ]:
excel_path = "productDataOptimization/data_GM.xlsx"
sheet_data = load_excel_data(excel_path)

In [ ]:
sheet_data.keys()

dict_keys(['AW 0.1.001', 'AW 01.005', 'AW 01.030', 'AW 01.033', 'AW 01.013'])

In [ ]:
sheet_data["AW 0.1.001"]["properties"]

{'Fassade': array([nan, 'D'], dtype=object),
 'Lattung': array(['Hinterlüftung', 'D'], dtype=object),
 'Trägerplatte': array([0.045, 'E'], dtype=object),
 'Konstruktion': array([0.13, 'D'], dtype=object),
 'Zellulosefaserdämmung': array([0.038, 'B'], dtype=object),
 'OSB': array([0.13, 'C'], dtype=object),
 'Unterkonstruktion': array([0.13, 'D'], dtype=object),
 'Mineralfaserdämmung': array([0.035, 'A2'], dtype=object),
 'GKB': array([0.32, 'A2'], dtype=object),
 'GKB2': array([0.32, 'A2'], dtype=object)}

In [ ]:
from itertools import product
def get_combinations(data, return_type = "index", sampling_method = None):
  #Logic of filter data now is if that layer has more than 5 variants, get only half of it.
  if return_type == "index":
    if sampling_method == "vertical":
      thickness_lists = [list(range(len(layer)))[::2] if len(layer)> 5 else list(range(len(layer))) for layer in data.values()]
    if sampling_method == "horizontal":
      thickness_lists = [list(range(len(layer)))[::2] if "Zellulosefaserdämmung" not in key else list(range(len(layer))) for key, layer in data.items()]
    if sampling_method == None:
      thickness_lists = [list(range(len(layer))) for layer in data.values()]
  else:
    if sampling_method == "vertical":
      thickness_lists = [layer[:, 0][::2] if len(layer)> 5 else layer[:, 0] for layer in data.values()]
    if sampling_method == "horizontal":
      thickness_lists = [layer[:, 0][::2] if "Zellulosefaserdämmung" not in key else layer[:, 0] for key, layer in data.items()]
  return np.array(list(product(*thickness_lists)))

In [ ]:
"konstruktion" in "Unterkonstruktion"

True

In [ ]:
factors = ["thickness", "price", "U_value", "fire_rating", "acoustics"]

def get_U_values(layers, thicknesses, lambdas, rsi = 0.13, rse = 0.04):
  omitted = ["Fassade", "Konstruktion", "Lattung", "Unterkonstruktion", "Putzfassade"]
  mask = [not any(keyword in l for keyword in omitted) for l in layers]
  #thickness/1000 because of lambda unit takes on meter
  partial_u = thicknesses.copy()[mask]/lambdas[mask].astype(float) / 1000
  return round(1/(rse + np.sum(partial_u) +rsi),5)

def get_overall_VKF(data):
  rfs = [int(props[1][-1]) for props in data["properties"].values()]
  #0. certain rules apply
  #1. if all layers take the same rf then the overall assembly has the same rf
  #2. if not, take the lowest rfs, programmtically we just always takes the lowest rfs
  #3. the rfs of the construction layer is detemriend by the layers before and after it, take the lowest rfs for the 2
  #4. Second thought the second logic is useless, cause at the end its the lowest one of the layers that matter. ??? Should ask

  for i, layer in enumerate(data["properties"].keys()):
    if "Konstruktion" in layer or "konstruktion" in layer:
      rf_prev = rfs[i-1]
      rf_next = rfs[i+1]
      # the bigger the value, the worse it is, idea is to take the worse
      if rf_prev > rf_next:
        rfs[i] = rf_prev
      else:
        rfs[i] = rf_next
  return max(rfs)


def get_performance_factors(combinations, data, factors):
  performances = np.zeros((len(combinations), len(factors)))
  properties = data["properties"]
  variants = data["variants"]
  layers = list(variants.keys())
  for i, combination in enumerate(combinations):
    thicknesses = np.array([variants[layers[i]][c][0] for i, c in enumerate(combination)])
    # overall thickness calculations excludes konstruction layer
    thickness_mask = [False if "Konstruktion" in props or "Unterkonstruktion" in props else True for props in properties.keys()]
    lambdas = np.array([ p[0] for p in properties.values()])
    U_value = get_U_values(layers, thicknesses, lambdas)
    row = performances[i]
    thickness = np.sum(thicknesses[thickness_mask])
    price = np.sum([variants[layers[i]][c][1] for i, c in enumerate(combination)])
    row[0], row[1], row[2] = thickness, price, U_value
  return performances

In [ ]:
@timeit
def get_all_perfromances(sheet_data, factors, sampling_method = None):
  total_combinations = 0
  for wall, data in sheet_data.items():
    print(wall)
    # For GM, we do not calculate the VKF
    # overall_VKF = get_overall_VKF(data)
    # print(overall_VKF)
    combinations = get_combinations(data["variants"], return_type= "index", sampling_method = sampling_method)
    performancees = get_performance_factors(combinations, data, factors)
    data["performances"] = performancees
    total_combinations += len(combinations)
  return

In [ ]:
get_all_perfromances(sheet_data, factors, sampling_method= None)

AW 0.1.001
AW 01.005
AW 01.030
AW 01.033
AW 01.013
⏱️ Function `get_all_perfromances` took 3.7225 seconds


In [ ]:
sheet_data["AW 0.1.001"]["performances"]

array([[3.1600e+02, 1.9400e+02, 1.6106e-01, 0.0000e+00, 0.0000e+00],
       [3.1850e+02, 1.9500e+02, 1.6086e-01, 0.0000e+00, 0.0000e+00],
       [3.1850e+02, 1.9500e+02, 1.6086e-01, 0.0000e+00, 0.0000e+00],
       ...,
       [5.2650e+02, 2.1700e+02, 9.2500e-02, 0.0000e+00, 0.0000e+00],
       [5.2650e+02, 2.1700e+02, 9.2500e-02, 0.0000e+00, 0.0000e+00],
       [5.2900e+02, 2.1800e+02, 9.2440e-02, 0.0000e+00, 0.0000e+00]])

In [ ]:
def get_global_bounds(sheet_data):
    all_thickness = []
    all_price = []
    all_u_value = []

    for key in sheet_data:
        data = sheet_data[key]["performances"]
        if data.shape[1] >= 3:  # Ensure at least 3 columns
            all_thickness.append(data[:, 0])
            all_price.append(data[:, 1])
            all_u_value.append(data[:, 2])

    # Concatenate all data
    thickness_all = np.concatenate(all_thickness)
    price_all = np.concatenate(all_price)
    u_value_all = np.concatenate(all_u_value)

    return {
        "thickness": (thickness_all.min(), thickness_all.max()),
        "price": (price_all.min(), price_all.max()),
        "u_value": (u_value_all.min(), u_value_all.max())
    }

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display

def interactive_3d_plot_3(sheet_data, t_tol=20, u_tol=0.05):
    look_keys = list(sheet_data.keys())
    min_max_global = get_global_bounds(sheet_data)

    # ---------- inner plotting callback ----------
    def update(key="Aussenwand 1.1",
               t_thresh=0, p_thresh=0, u_thresh=0,
               t_tol=t_tol, u_tol=u_tol):

        data_array = sheet_data[key]["performances"]
        thickness, price, u_value = data_array[:, 0], data_array[:, 1], data_array[:, 2]

        mask = (np.abs(thickness - t_thresh) <= t_tol) & \
               (price <= p_thresh) & \
               (np.abs(u_value - u_thresh) <= u_tol)

        good = data_array[mask]
        bad  = data_array[~mask]

        # ------------ start figure ---------------
        fig = plt.figure(figsize=(11, 9))
        ax  = fig.add_subplot(111, projection='3d')

        # Main scatter
        ax.scatter(good[:, 0], good[:, 1], good[:, 2],
                   c='green', label='Meets req', zorder=2)
        ax.scatter(bad[:, 0],  bad[:, 1],  bad[:, 2],
                   c='red',   label='Fails req', zorder=1)

        # ----- highlight min & max price among valid points -----
        if good.size:
            min_idx, max_idx = np.argmin(good[:, 1]), np.argmax(good[:, 1])
            pt_min, pt_max   = good[min_idx], good[max_idx]

            h_min = ax.scatter(*pt_min[:3], c='yellow', s=90,
                               marker='o', edgecolors='k', zorder=5,
                               label=f"Min CHF {pt_min[1]:.0f}")
            h_max = ax.scatter(*pt_max[:3], c='blue', s=90,
                               marker='o', edgecolors='k', zorder=5,
                               label=f"Max CHF {pt_max[1]:.0f}")

        # Threshold guide lines
        ax.plot([t_thresh]*2, [price.min(), price.max()], [u_thresh]*2, 'r--', zorder=0)
        ax.plot([thickness.min(), thickness.max()], [p_thresh]*2, [u_thresh]*2, 'r--', zorder=0)
        ax.plot([t_thresh]*2, [p_thresh]*2, [u_value.min(), u_value.max()], 'r--', zorder=0)

        ax.set_xlabel("Thickness [mm]")
        ax.set_ylabel("Price [CHF/m²]")
        ax.set_zlabel("U-Value [W/m²K]")
        ax.set_title(f"Performance – {key}")
        ax.legend(loc='upper right')
        plt.show()
    # ----------- widgets ----------------
    key_dropdown = widgets.Dropdown(options=look_keys, description='Aussenwand')

    t_slider = widgets.IntSlider(
        value=250, min=int(min_max_global["thickness"][0]),
        max=int(min_max_global["thickness"][1]), step=10,
        description='Tar_Thickness')

    p_slider = widgets.IntSlider(
        value=int(min_max_global["price"][1]), min=int(min_max_global["price"][0]),
        max=int(min_max_global["price"][1]), step=10,
        description='Max_Price')

    u_slider = widgets.FloatSlider(
        value=0.20, min=round(float(min_max_global["u_value"][0]), 3),
        max=round(float(min_max_global["u_value"][1]), 2), step=0.005,
        description='Tar_U-Value')

    t_tol_slider = widgets.IntSlider(value=t_tol, min=0, max=50, step=5,  description='t_tol')
    u_tol_slider = widgets.FloatSlider(value=u_tol, min=0, max=0.3, step=0.01, description='u_tol')

    display(widgets.interactive(
        update,
        key        = key_dropdown,
        t_thresh   = t_slider,
        p_thresh   = p_slider,
        u_thresh   = u_slider,
        t_tol      = t_tol_slider,
        u_tol      = u_tol_slider
    ))


In [ ]:
interactive_3d_plot_3(sheet_data, t_tol=10, u_tol=0.05)

interactive(children=(Dropdown(description='Aussenwand', options=('AW 0.1.001', 'AW 01.005', 'AW 01.030', 'AW …

# Sampling

In [ ]:
get_all_perfromances(sheet_data, factors, sampling_method= "horizontal")

AW 0.1.001
AW 01.005
AW 01.030
AW 01.033
AW 01.013
⏱️ Function `get_all_perfromances` took 0.1248 seconds


In [ ]:
interactive_3d_plot_3(sheet_data, t_tol=10, u_tol=0.05)

interactive(children=(Dropdown(description='Aussenwand', options=('AW 0.1.001', 'AW 01.005', 'AW 01.030', 'AW …